In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGCHAIN_TRACKING_V2"] = "true"


In [ ]:
#model only
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")
result = llm.invoke("what is the capital of india")
print(result.content)


The capital of India is New Delhi.


In [12]:
from langchain_groq import ChatGroq
model = ChatGroq(model="compound-beta-mini")
model.invoke("who is the chief minister of kerala,india")

AIMessage(content='The current Chief Minister of Kerala, India is Pinarayi Vijayan. He has been serving as the Chief Minister since May 25, 2016, and was re-elected for a second term on May 20, 2021. He is a member of the Communist Party of India (Marxist) and represents the Dharmadom constituency in the Kerala Legislative Assembly.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 3276, 'total_tokens': 3396, 'completion_time': 0.303455, 'prompt_time': 0.093003, 'queue_time': 0.100619, 'total_time': 0.396457}, 'model_name': 'compound-beta-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--7bc26ba4-f6f3-482e-88e4-df615f406184-0', usage_metadata={'input_tokens': 3276, 'output_tokens': 120, 'total_tokens': 3396})

In [ ]:
# prompt + model
#PROMPT Engineering
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Provide short answers to user queries"),
        ("user", "{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Provide short answers to user queries'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [11]:
chain = prompt|llm
response = chain.invoke({"input":"who is the president of iran "})
print(response.content)


The current President of Iran is Masoud Pezeshkian. He has been in office since July 28, 2024, after winning the 2024 presidential election.


In [13]:
# prompt + model + outputParser

#outputParser
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt|model|output_parser

response = chain.invoke({"input":"are you a human or not"})
print(response)

No, I am not human. I'm a computer program designed to simulate conversations and answer questions to the best of my ability. I'm an AI, specifically a large language model, built by Groq.


1️⃣ prompt: builds the input (prompt) for the model.
2️⃣ model: generates an output (structured response).
3️⃣ output_parser: parses the output to a string.

In [17]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
output_parser.get_format_instructions()


'Return a JSON object.'

In [21]:

from langchain_core.prompts import PromptTemplate #Prompt
from langchain_groq import ChatGroq #Model
from langchain_core.output_parsers import JsonOutputParser #OuptutParser

#Step-1 : Prompt
#prompt - not chatPromptTemplate, but PromptTemplate
prompt = PromptTemplate(
    template="Answer user query \n Return a JSON object. \n {query} \n ",
    input_variables=["query"]
)

#Step-2 : Model 
model = ChatGroq(model="compound-beta-mini") 

#Step-3 : Output Parser
output_parser = JsonOutputParser()

#Step-4 : Chaining
chain = prompt|model|output_parser

#Step-5 : Response
response = chain.invoke({"query":"who is the first prime minister of india"})

print(response)



{'answer': 'Jawaharlal Nehru', 'additional_info': {'term': 'First Prime Minister of India', 'tenure': 'August 15, 1947 - May 27, 1964'}}


In [22]:
#in standard, we manipulate prompt like

from langchain_core.prompts import PromptTemplate #Prompt
from langchain_groq import ChatGroq #Model
from langchain_core.output_parsers import JsonOutputParser #OuptutParser

#Step-1 : Output Parser
output_parser = JsonOutputParser()

#Step-2 : Model 
model = ChatGroq(model="compound-beta-mini") 

#Step-3 : Prompt
prompt = PromptTemplate(
    template="Answer user query \n {format_instruction} \n {query} \n ",
    input_variables=["query"],
    partial_variables={"format_instruction":output_parser.get_format_instructions()}
)

#Step-4 : Chaining
chain = prompt|model|output_parser

#Step-5 : Response
response = chain.invoke({"query":"who is the first prime minister of india"})

print(response)

{'name': 'Jawaharlal Nehru', 'designation': 'First Prime Minister of India', 'term': '1947-1964'}


In [24]:
# chatPromptTemplate : it is like an conversational prompt .systems says, user says, then sytem, then user.....
# PromptTemplate : single converstation

#using chatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","answer user queries in json format "),
        ("user","{query}")
    ]
)
response = chain.invoke({"query":"who is the goat of football"})
print(response)


{'query': 'who is the goat of football', 'response': {'GOAT of Football': 'Lionel Messi', 'Reason': "Lionel Messi is widely regarded as the Greatest of All Time (GOAT) in football, with numerous accolades and records, including winning the World Cup in 2022 and multiple Ballon d'Or awards.", 'Other Contenders': ['Cristiano Ronaldo', 'Pelé', 'Diego Maradona', 'Johan Cruyff', 'Zinedine Zidane', 'Michel Platini', 'Alfredo Di Stefano'], 'Context': 'The debate about the GOAT of football is ongoing, with various opinions and perspectives. However, based on recent trends and expert opinions, Lionel Messi is often considered the GOAT.'}}
